In [1]:
import pandas as pd
import numpy as np
import csv
import re
from langdetect import detect
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
import time #go to bottom for working code

In [2]:
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+' # For removing mentions.
pat2 = r'https?://[^ ]+' # For removing links.
combined_pat = r'|'.join((pat1, pat2)) # For removing links.
www_pat = r'www.[^ ]+' # For removing links.
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"} # For removing negations.
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b') # For removing negations.

def tweet_cleaner(text):
      """ This cleans the tweets text content
          so it can be machine readable.
      """
      soup = BeautifulSoup(text, 'lxml') # Decodes HTML.
      souped = soup.get_text()
      try: # UTF decoding.
          bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
      except:
          bom_removed = souped
      stripped = re.sub(combined_pat, '', bom_removed) # Gets rid of links.
      stripped = re.sub(www_pat, '', stripped) # Gets rid of links.
      lower_case = stripped.lower()
      neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
      letters_only = re.sub("[^a-zA-Z]", " ", neg_handled) # Only lets letters in.
      words = [x for x  in tok.tokenize(letters_only) if len(x) > 1] # Below removes whitespace
      return (" ".join(words)).strip()

In [4]:
df = pd.read_csv('tesla_cleaned.csv', engine='python') #read in csv

In [5]:
length_of_df = len(df)
print(length_of_df)
df.head()

752294


,Time,Likes,Retweets,Text
0,2019-01-01 0:00,54,2,Proud papa! #newcar #tesla https://t.co/4YXQwp...
1,2019-01-01 0:00,1,0,"""iPhone XS Max Patladı""\n\nhttps://t.co/KU6Edn..."
2,2019-01-01 0:00,0,0,Sofocast: Leevi Parsama puhuu fiksuja: käytä k...
3,2019-01-01 0:06,0,0,#TeamElon #Tesla #Model3 ❤🚘❤ https://t.co/AfJK...
4,2019-01-01 0:07,0,0,#TESLA : $TSLA Dec-31 Update #StockMarket #Tec...


In [21]:
def cleanTxt(text):
    text = re.sub(r'@[A-Za-z0–9]+', '', text) #Removing @mentions
    text = re.sub(r'#', '', text) # Removing '#' hash tag
    text = re.sub(r'RT[\s]+', '', text) # Removing RT
    text = re.sub(r'https?:\/\/\S+', '', text) # Removing hyperlink

In [22]:
tic = time.perf_counter()

#for index, row in df.iterrows():
    #row["Text"] = tweet_cleaner(row["Text"]) #why tf doesn't this work 
    
#for index, row in df.iterrows():
    #row["Text"] = row["Text"].apply(tweet_cleaner) 


    
toc = time.perf_counter()
print(f"Cleaned {len(df)} in {toc - tic:0.4f} seconds")

AttributeError: 'str' object has no attribute 'apply'

In [15]:
df.head(10) #ceck random range of dataset to ensure it was properly cleaned

,Time,Likes,Retweets,Text
0,2019-01-01 0:00,54,2,Proud papa! #newcar #tesla https://t.co/4YXQwp...
1,2019-01-01 0:00,1,0,"""iPhone XS Max Patladı""\n\nhttps://t.co/KU6Edn..."
2,2019-01-01 0:00,0,0,Sofocast: Leevi Parsama puhuu fiksuja: käytä k...
3,2019-01-01 0:06,0,0,#TeamElon #Tesla #Model3 ❤🚘❤ https://t.co/AfJK...
4,2019-01-01 0:07,0,0,#TESLA : $TSLA Dec-31 Update #StockMarket #Tec...
5,2019-01-01 0:08,0,0,#TeslaModel3 #Tesla #ElonMusk #Autopilot #Tran...
6,2019-01-01 0:10,0,0,if my memory serves me correct #tesla will be ...
7,2019-01-01 0:11,1,0,#tesla #mall #cars #batterydriven #stateofthea...
8,2019-01-01 0:14,3,0,Installed this bad boy today in the garage! No...
9,2019-01-01 0:14,0,0,Tesla #Tesla U.S. #US #Hybrid &amp; Alternativ...


In [17]:
#df.drop('Source', inplace=True, axis=1)
#df.drop('Unnamed: 0', inplace=True, axis=1)
#df.drop('tweet_id', inplace=True, axis=1)
#df.drop('User location', inplace = True, axis = 1) #delete columns we don't need

In [20]:
type(df['Text'][1])

str

In [ ]:
j = len(df)
i = 0 #remove 
for index, row in df.iterrows():
    if detect(df["tweet"][row])!= 'en': #this should work?
        df.drop(index, inplace=True)
print(j - len(df), "rows removed") #print tumber of rows that got removed

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [ ]:
def sentimentScore(Tweet):
    analyser = SentimentIntensityAnalyzer()
    score_values = []
    for tweet in df['tweet']:
        vs1 = analyzer.polarity_scores(tweet)
        score_values.append(vs1)
    return score_values

In [ ]:
sentiment_scores = pd.DataFrame(sentimentScore(df['tweet']))
df.head()

In [23]:
#Mustafas code

In [3]:
import re
import pandas as pd
import urllib3
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
import time
from itertools import islice
import analyzer

In [7]:
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9_]+' # For removing mentions.
pat2 = r'https?://[^ ]+' # For removing links.
combined_pat = r'|'.join((pat1, pat2)) # For removing links.
www_pat = r'www.[^ ]+' # For removing links.
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"} # For removing negations.
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b') # For removing negations.
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')
def tweet_cleaner(text):
      """ This cleans the tweets text content
          so it can be machine readable.
      """
      soup = BeautifulSoup(text, 'lxml') # Decodes HTML.
      souped = soup.get_text()
      try: # UTF decoding.
          bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
      except:
          bom_removed = souped
      stripped = re.sub(combined_pat, '', bom_removed) # Gets rid of links.
      stripped = re.sub(www_pat, '', stripped) # Gets rid of links.
      lower_case = stripped.lower()
      neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
      letters_only = re.sub("[^a-zA-Z]", " ", neg_handled) # Only lets letters in.
      words = [x for x  in tok.tokenize(letters_only) if len(x) > 1] # Below removes whitespace
      return (" ".join(words)).strip()

In [4]:
tic = time.perf_counter()

df = pd.read_csv("teslaFullDataset.csv", engine= "python")


toc = time.perf_counter()

print(f"Cleaned {len(df)} in {toc - tic:0.4f} seconds")
df.head()

Cleaned 767393 in 8.1074 seconds


,Unnamed: 0,Like count,Retweet count,Source,User location,date,tweet,tweet_id
0,0,1.0,0.0,Twitter Web Client,Chicago,2019-02-28 23:40:44,"With many stores closing, going to be harder t...",1.10126597703825e+18
1,1,0.0,0.0,Twitter for iPhone,Manchester,2019-02-28 23:38:48,"#Tesla drives down price for Model 3 to $35,00...",1.1012654913504379e+18
2,2,0.0,0.0,Twitter for iPhone,"iPhone: 37.893063,-85.541092",2019-02-28 23:50:31,@elonmusk I feel like everyone is talking abou...,1.1012684388136919e+18
3,3,0.0,0.0,Instagram,"San Francisco, CA",2019-02-28 23:54:06,Pixie is taking a liking to #tesla #dogslife #...,1.1012693404631613e+18
4,4,0.0,0.0,Twitter for iPad,Long Island,2019-02-28 23:52:55,@GerberKawasaki @elonmusk ... and #Tesla’s har...,1.101269044919976e+18


In [5]:
df.drop('Source', inplace=True, axis=1)
df.drop('Unnamed: 0', inplace=True, axis=1)
df.drop('tweet_id', inplace=True, axis=1)
df.drop('User location', inplace = True, axis = 1) #delete columns we don't need
df.head()

,Like count,Retweet count,date,tweet
0,1.0,0.0,2019-02-28 23:40:44,"With many stores closing, going to be harder t..."
1,0.0,0.0,2019-02-28 23:38:48,"#Tesla drives down price for Model 3 to $35,00..."
2,0.0,0.0,2019-02-28 23:50:31,@elonmusk I feel like everyone is talking abou...
3,0.0,0.0,2019-02-28 23:54:06,Pixie is taking a liking to #tesla #dogslife #...
4,0.0,0.0,2019-02-28 23:52:55,@GerberKawasaki @elonmusk ... and #Tesla’s har...


In [11]:
tic = time.perf_counter()

data = []
for index, row in df.iterrows():
    data.append(tweet_cleaner(str(row["tweet"])))
df["Clean"] = data
df.to_csv("Tesla_Final.csv")

toc = time.perf_counter()
print(f"Cleaned {len(df)} in {toc - tic:0.4f} seconds")

Cleaned 767393 in 280.3766 seconds


In [12]:
df[234000:234550]

,Like count,Retweet count,date,tweet,Clean
234000,0.0,0.0,2019-09-15 18:50:59,Imagine watching Netflix while your car is par...,imagine watching netflix while your car is par...
234001,0.0,0.0,2019-09-15 17:57:53,"For those, who recently bought a #Model3 or ot...",for those who recently bought model or other e...
234002,2.0,0.0,2019-09-15 18:56:06,Raise your hand if you're a college student wh...,raise your hand if you re college student whos...
234003,1.0,0.0,2019-09-15 18:39:26,Not a bad turnout for one of the first UK Tesl...,not bad turnout for one of the first uk tesla ...
234004,0.0,0.0,2019-09-15 18:45:56,American capitalists profit off our Chinese ta...,american capitalists profit off our chinese ta...
...,...,...,...,...,...
234545,0.0,0.0,2019-09-14 19:03:44,Just crossed 5000 miles on our 11 States road ...,just crossed miles on our states road so far n...
234546,4.0,1.0,2019-09-14 19:29:20,Neben vielen interessanten Informationen und n...,neben vielen interessanten informationen und n...
234547,0.0,0.0,2019-09-14 18:40:01,Tell @E_l_o_n I want an internship spacex 🚀 \n...,tell want an internship spacex you must learn ...
234548,0.0,0.0,2019-09-14 18:55:33,This:\nBrian from the YouTube channel i1:\n“Yo...,this brian from the youtube channel your car n...


In [122]:
tic = time.perf_counter()
j = len(df)
length = 0


for index, row in islice(df.iterrows(), index, None):
    length = row
    if detect(row["Clean"]) == "error":
        df.drop(index, inplace=True)
        print("row", row, "is not able to be determined")
    elif detect(row["Clean"])!= 'en': #this should work?
        df.drop(index, inplace=True)

print(length)
toc = time.perf_counter()
print(j - len(df), "rows removed in", toc - tic, "sec") #print tumber of rows that got removed


0
0 rows removed in 40.42861689900019 sec


In [121]:
#LangDetectException: No features in text.
print(row, "\n")
print(index, "\n")
df.drop(index, inplace = True)

Like count                                     4.0
Retweet count                                  0.0
date                           2020-12-22 04:10:58
tweet            https://t.co/W56bCCVz9f\n\n#Tesla
Clean                                             
Name: 744973, dtype: object 

744973 



In [123]:
df.to_csv("Tesla_Final_Data.csv")

In [4]:
df = pd.read_csv("Tesla_Final_Data.csv")

In [5]:
df.head(10)

,Unnamed: 0,Like count,Retweet count,date,tweet,Clean
0,0,1.0,0.0,2019-02-28 23:40:44,"With many stores closing, going to be harder t...",with many stores closing going to be harder to...
1,1,0.0,0.0,2019-02-28 23:38:48,"#Tesla drives down price for Model 3 to $35,00...",tesla drives down price for model to
2,2,0.0,0.0,2019-02-28 23:50:31,@elonmusk I feel like everyone is talking abou...,feel like everyone is talking about the car bu...
3,3,0.0,0.0,2019-02-28 23:54:06,Pixie is taking a liking to #tesla #dogslife #...,pixie is taking liking to tesla dogslife trave...
4,5,0.0,0.0,2019-02-28 23:41:50,"@Bobocr1 @skabooshka Based on the M3 Tracker, ...",based on the tracker tesla only needs to avera...
5,6,0.0,0.0,2019-02-28 23:43:26,#Tesla shares drop after Elon #Musk says he do...,tesla shares drop after elon musk says he does...
6,7,30.0,3.0,2019-02-28 23:53:51,Dramatic changes coming to #Tesla Sales! Focus...,dramatic changes coming to tesla sales focus s...
7,9,5.0,0.0,2019-02-28 23:54:38,Maybe by the time I'm in the market for a new ...,maybe by the time in the market for new car wi...
8,10,1.0,0.0,2019-02-28 23:43:18,We continue have no position in $TSLA but I’m ...,we continue have no position in tsla but sorry...
9,12,0.0,0.0,2019-02-28 23:53:53,$tsla $tslaq #Tesla\n\nTesla Slashing Prices a...,tsla tslaq tesla tesla slashing prices and clo...


In [6]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import twitter_samples 
 
nltk.download('twitter_samples')
nltk.download('vader_lexicon')
 
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/griffinclark/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/griffinclark/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [7]:
i = 0
neg = []
neu = []
pos = []
analysis = []

for i in range(len(df)-1):
    try:
        analysis.append(analyzer.polarity_scores(df["Clean"][i]))
    except:
        analysis.append({'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0})
        i+=1
        
    #['neg']
    #neu[i] = analyzer.polarity_scores(df["Clean"][i])#['neu']
    #pos[i] = analyzer.polarity_scores(df["Clean"][i])#['pos']
    

    
#df['neg'] = df['tweets'].apply(lambda x:analyzer.polarity_scores(x)['neg'])
#df['neu'] = df['tweets'].apply(lambda x:analyzer.polarity_scores(x)['neu'])
#df['pos'] = df['tweets'].apply(lambda x:analyzer.polarity_scores(x)['pos'])
#df['compound'] = df['tweets'].apply(lambda x:analyzer.polarity_scores(x)['compound'])

In [8]:
analysis

[{'neg': 0.103, 'neu': 0.897, 'pos': 0.0, 'compound': -0.3182},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.124, 'neu': 0.798, 'pos': 0.078, 'compound': -0.2617},
 {'neg': 0.0, 'neu': 0.803, 'pos': 0.197, 'compound': 0.4019},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.104, 'neu': 0.644, 'pos': 0.252, 'compound': 0.4588},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.121, 'neu': 0.733, 'pos': 0.147, 'compound': 0.5053},
 {'neg': 0.208, 'neu': 0.792, 'pos': 0.0, 'compound': -0.2732},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.655, 'pos': 0.345, 'compound': 0.7003},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.813, 'pos': 0.187, 'compound': 0.3182},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, '

In [9]:
len(analysis)

735542

In [22]:
i = 0
for i in range(len(analysis)-1):
    df['neg'][i] = analysis[i]['neg']
    df['neu'][i] = analysis[i]['neu']
    df['pos'][i] = analysis[i]['pos']
    df['compound'][i] = analysis[i]['compound']

<ipython-input-22-900ea5fd1586>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['neg'][i] = analysis[i]['neg']
<ipython-input-22-900ea5fd1586>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['neu'][i] = analysis[i]['neu']
<ipython-input-22-900ea5fd1586>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pos'][i] = analysis[i]['pos']
<ipython-input-22-900ea5fd1586>:6: SettingWithCopyWarning: 
A value is trying to be set on a c

In [24]:
df[1422:1430]

,Unnamed: 0,Like count,Retweet count,date,tweet,Clean,neg,neu,pos,compound
1422,2160,0.0,0.0,2019-02-27 09:54:05,🤔 Could be refuled #electricity on go the same...,could be refuled electricity on go the same wa...,0.000,1.000,0.000,0.0000
1423,2161,2.0,0.0,2019-02-27 09:08:28,#Tesla is expecting a radical improvement in A...,tesla is expecting radical improvement in auto...,0.000,0.798,0.202,0.7964
1424,2162,33.0,14.0,2019-02-27 10:25:42,Volkswagen has signed on with #Tesla to utiliz...,volkswagen has signed on with tesla to utilize...,0.000,1.000,0.000,0.0000
1425,2163,0.0,0.0,2019-02-27 09:48:08,New #Tesla fiery incident: how could they comm...,new tesla fiery incident how could they commer...,0.118,0.882,0.000,-0.3400
1426,2165,1.0,0.0,2019-02-27 10:30:00,Legal experts say it's unlikely #ElonMusk will...,legal experts say it unlikely elonmusk will fa...,0.170,0.796,0.034,-0.6059
1427,2168,0.0,0.0,2019-02-27 09:20:47,iOS 12 and Siri and Today at Apple - 9to5 Mac ...,ios and siri and today at apple to mac is my f...,0.000,0.904,0.096,0.4939
1428,2169,0.0,0.0,2019-02-27 09:30:08,"Elon Musk, the CEO Tesla, was questioned by SE...",elon musk the ceo tesla was questioned by sec ...,0.219,0.781,0.000,-0.8271
1429,2171,0.0,0.0,2019-02-27 09:30:55,"Elon Musk contro la Sec, si riaprono le ostili...",elon musk contro la sec si riaprono le ostilit...,0.000,1.000,0.000,0.0000
